In [1]:


#0) instalar todas as dependencias
%pip install langchain langchain-community openai pandas pandasql python-dotenv


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Pichau\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#1) Importaçao das bibliotecas necessárias
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool
from dotenv import load_dotenv
import os
import zipfile
import pandas as pd
from pandasql import sqldf
from io import TextIOWrapper
import re 

#import pandasql as ps
#from openai import OpenAI
#from langchain_core.tools import Tool








# Variável global para armazenar o dataframe
df_global = None

#2) Criação da função das ferramentas
def carregar_arquivos(arquivo_zip_path: str):
    """
    **PRIMEIRA ETAPA SEMPRE**
    Quando necessário extrai os arquivos do formato .zip (Extrair,Extract,Descompactar,Unzip,Abrir,Open,Deszipar,Decompress,Desempacotar,Unarchive,Desarquivar)
    Exemplo: 'C:\\Users\Pichau\Downloads\\teste\\202401_NFs.zip'
    Exemplo2: 'C:\\Users\\ABC\\Documents\\arquivo.zip'
    Exemplo3: 'C:\\_______NOME.zip'
    """
    arquivo_zip_path = arquivo_zip_path.strip("'\"")
    global df_global # Declara que você vai modificar a variável global
    grupo_csv = {}
    
    with zipfile.ZipFile(arquivo_zip_path, 'r') as zip_ref:
        for arquivos in zip_ref.namelist():
            if arquivos.endswith('.csv'):
                with zip_ref.open(arquivos) as arq:
                    # Adicionado errors='ignore' para tentar lidar com problemas de codificação
                    df = pd.read_csv(TextIOWrapper(arq,'utf-8', errors='ignore'))
                    grupo_csv[arquivos] = df
            else:
                print(f"Sem suporte para o arquivo: '{arquivos}' não é um arquivo .csv")


        sorted_keys = sorted(grupo_csv.keys())
        primeiro_df = grupo_csv[sorted_keys[0]]
        segundo_df = grupo_csv[sorted_keys[1]]
       
        df_global = pd.merge(primeiro_df, segundo_df, on='CHAVE DE ACESSO', how='left')
        df_global.columns = df_global.columns.str.replace('_x$', '', regex=True)
        df_global = df_global.drop(columns=['MODELO_y','SÉRIE_y','NÚMERO_y','NATUREZA DA OPERAÇÃO_y','DATA EMISSÃO_y','CPF/CNPJ Emitente_y','RAZÃO SOCIAL EMITENTE_y','INSCRIÇÃO ESTADUAL EMITENTE_y','UF EMITENTE_y','MUNICÍPIO EMITENTE_y','CNPJ DESTINATÁRIO_y','NOME DESTINATÁRIO_y','UF DESTINATÁRIO_y','INDICADOR IE DESTINATÁRIO_y','DESTINO DA OPERAÇÃO_y','CONSUMIDOR FINAL_y','PRESENÇA DO COMPRADOR_y'])


        return f"Arquivos carregados e merged com sucesso! DataFrame pronto para análise com {len(df_global)} linhas e {len(df_global.columns)} colunas. As colunas disponíveis são: {list(df_global.columns)}."

def consulta_sql(query: str) -> str:
    """
    **SEGUNDA ETAPA SEMPRE**
    Executa uma consulta SQL no DataFrame df_global.
    """
    global df_global
    if df_global is None:
        return "O DataFrame global ainda não foi carregado."

    try:
        result = sqldf(query, {'df_global': df_global})
        return result.to_string(index=False)
    except Exception as e:
        return f"Erro na consulta: {e}"
    

def mapear_colunas(args: dict) -> str:
    """
    **TERCEIRA ETAPA OPCIONAL**
    Caso tenha dificuldades em encontrar o nome da coluna adequadamente, utilize essa ferramenta que mapeia os nomes das colunas do DataFrame global (df_global) para nomes mais simples e padronizados.
    Essa ferramenta deve ser usada quando a consulta SQL falhar por nomes de colunas incorretos ou complexos.
    """
    import re
    
    global df_global
    if df_global is None:
        return "O DataFrame global (df_global) ainda não foi carregado."

    colunas_originais = df_global.columns.tolist()
    
    # Cria dicionário de mapeamento simples
    mapeamento = {}
    for col in colunas_originais:
        col_nova = col.upper()
        col_nova = re.sub(r"[^A-Z0-9]", "_", col_nova)  # Substitui espaços/símbolos por _
        col_nova = re.sub(r"_+", "_", col_nova).strip("_")  # Remove múltiplos _
        mapeamento[col] = col_nova

    # Aplica o mapeamento
    df_global.columns = [mapeamento[col] for col in df_global.columns]
    
    retorno = "\n".join([f"{orig} -> {novo}" for orig, novo in mapeamento.items()])
    return f"Colunas renomeadas com sucesso:\n{retorno}"    

#3) Criando a ferramenta
zip_tool = Tool(
    name="carregar_arquivos",
    func=carregar_arquivos,
    description=carregar_arquivos.__doc__.strip() # Pega a docstring da função como descrição,
)

query_tool = Tool(
name="query_arquivo",
    func=consulta_sql,
    description=consulta_sql.__doc__.strip() # Pega a docstring da função como descrição,
)

map_tool = Tool(
name='mapear_colunas',
func=mapear_colunas,
description=mapear_colunas.__doc__.strip() # Pega a docstring da função como descrição

)


In [3]:

load_dotenv() #carrega o segredo

# 4) Escolhe o modelo selecionado e configuração de temperatura
llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    base_url="https://openrouter.ai/api/v1",
    temperature=0.25,
    api_key = os.getenv("api_key"),
    max_tokens=500
)

   
# 5) Inicializando o agente com a ferramenta
agent = initialize_agent(
    tools=[zip_tool,query_tool,map_tool],
    llm=llm,
    agent_type = AgentType.OPENAI_MULTI_FUNCTIONS,
    verbose=True,
    handle_parsing_errors=True
)



C:\Users\Pichau\AppData\Local\Temp\ipykernel_6368\717820186.py:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
C:\Users\Pichau\AppData\Local\Temp\ipykernel_6368\717820186.py:14: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/doc

In [4]:
#6) Recebe a pergunta do usuário
user_input = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' Qual o menor valor da coluna 'Valor total'?"
# Executa o agente passando a pergunta
resposta = agent.run(user_input)

print(f"Resposta: {resposta}")

C:\Users\Pichau\AppData\Local\Temp\ipykernel_6368\4289259528.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = agent.run(user_input)




> Entering new AgentExecutor chain...
I need to start by extracting the contents of the ZIP file located at 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip'. This will allow me to access the data inside it for further analysis.

Action: carregar_arquivos  
Action Input: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip'  
Observation: Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', '

In [5]:
#6.1) Recebe a pergunta do usuário
user_input1 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' Qual o maior valor da coluna 'Valor total'?"
# Executa o agente passando a pergunta
resposta = agent.run(user_input1)

print(f"Resposta: {resposta}")



> Entering new AgentExecutor chain...
I need to extract the contents of the specified ZIP file before I can analyze the data within it. Therefore, I will use the `carregar_arquivos` action to extract the files.

Action: carregar_arquivos  
Action Input: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip'  
Observation: Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDAD

In [6]:
#6.2) Recebe a pergunta do usuário
user_input1 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' Qual a 'Razao social' que teve o segundo maior valor da coluna 'Valor total'?"
# Executa o agente passando a pergunta
resposta = agent.run(user_input1)

print(f"Resposta: {resposta}")



> Entering new AgentExecutor chain...
I need to extract the contents of the specified ZIP file first to access the data inside it. 

Action: carregar_arquivos  
Action Input: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip'  
Observation: Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDADE', 'UNIDADE', 'VALOR UNITÁRIO', 'VALOR TOTAL'].
Thought:I need to find the sec

In [7]:
#6.3) Recebe a pergunta do usuário
user_input1 = "Extraia o arquivo localizado em: 'C:\\Users\\Pichau\\Downloads\\teste\\202401_NFs.zip' Qual 'Tipo de produto' teve maior 'quantidade'?"
# Executa o agente passando a pergunta
resposta = agent.run(user_input1)

print(f"Resposta: {resposta}")



> Entering new AgentExecutor chain...
I need to extract the contents of the specified ZIP file first to access the data within it. 

Action: carregar_arquivos  
Action Input: 'C:\Users\Pichau\Downloads\teste\202401_NFs.zip'  
Observation: Arquivos carregados e merged com sucesso! DataFrame pronto para análise com 565 linhas e 30 colunas. As colunas disponíveis são: ['CHAVE DE ACESSO', 'MODELO', 'SÉRIE', 'NÚMERO', 'NATUREZA DA OPERAÇÃO', 'DATA EMISSÃO', 'EVENTO MAIS RECENTE', 'DATA/HORA EVENTO MAIS RECENTE', 'CPF/CNPJ Emitente', 'RAZÃO SOCIAL EMITENTE', 'INSCRIÇÃO ESTADUAL EMITENTE', 'UF EMITENTE', 'MUNICÍPIO EMITENTE', 'CNPJ DESTINATÁRIO', 'NOME DESTINATÁRIO', 'UF DESTINATÁRIO', 'INDICADOR IE DESTINATÁRIO', 'DESTINO DA OPERAÇÃO', 'CONSUMIDOR FINAL', 'PRESENÇA DO COMPRADOR', 'VALOR NOTA FISCAL', 'NÚMERO PRODUTO', 'DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'NCM/SH (TIPO DE PRODUTO)', 'CFOP', 'QUANTIDADE', 'UNIDADE', 'VALOR UNITÁRIO', 'VALOR TOTAL'].
Thought:Agora que extraí os ar